In [1]:
from inference import SparseModel

/home/generall/projects/vector_search/tmp/msmacro/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SparseModel()

/home/generall/projects/vector_search/tmp/msmacro/venv/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
QDRANT_URL='http://localhost:6333'
QDRANT_API_KEY=''
COLLECTION_NAME="quora"

In [4]:
from qdrant_client import QdrantClient, models
import math

2024-03-16 12:36:45.095 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


In [5]:
def rescore_vector(vector: dict) -> dict:
    new_vector = {}

    sorted_vector = sorted(vector.items(), key=lambda x: x[1], reverse=True)

    for num, (idx, _value) in enumerate(sorted_vector):
        new_vector[idx] = 1. # math.log(1./(num + 1) + 1.) # * value
    return new_vector


In [6]:
def conver_sparse_vector(sparse_vector: dict) -> models.SparseVector:
    indices = []
    values = []

    for (idx, value) in sparse_vector.items():
        indices.append(int(idx))
        values.append(value)

    return models.SparseVector(
        indices=indices,
        values=values
    )

In [7]:
client = QdrantClient(QDRANT_URL, api_key=QDRANT_API_KEY)

/home/generall/projects/vector_search/tmp/msmacro/venv/lib/python3.11/site-packages/qdrant_client/qdrant_remote.py:122: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")


In [8]:
query = "Do you think of mathematics as art or science?"
sparse_vector = rescore_vector(next(model.encode([query])))
sparse_vector_qdrant = conver_sparse_vector(sparse_vector)

limit = 20
result = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=models.NamedSparseVector(
        name="attention",
        vector=sparse_vector_qdrant
    ),
    with_payload=True,
    with_vectors=True,
    limit=limit
)

In [9]:
for idx, hit in enumerate(result):
    print(idx, hit)

0 id=435817 version=19847 score=4.03919 payload={'text': 'What are the similarities between philosophy of science and philosophy of mathematics?'} vector={'attention': SparseVector(indices=[1029, 1996, 1997, 1998, 2024, 2054, 2090, 2671, 4695, 5597, 12319], values=[0.0001106661, 0.06690782, 0.1594394, 0.09184596, 0.09434017, 0.056945104, 0.16237697, 1.140898, 1.1122377, 2.7387416, 0.87275857])} shard_key=None
1 id=107780 version=4898 score=3.9258626 payload={'text': 'What word could you use to replace Aesthete not for art but for science?'} vector={'attention': SparseVector(indices=[1029, 2000, 2005, 2017, 2021, 2025, 2054, 2063, 2071, 2224, 2396, 2671, 2773, 3367, 5672, 27065, 29347], values=[8.381888e-05, 0.055624176, 0.19380781, 0.09614431, 0.1876321, 0.12544827, 0.030799694, 0.20959824, 0.13683692, 0.12466651, 2.6887364, 1.140898, 0.36515567, 0.49572015, 0.38214764, 1.1065063, 1.254793])} shard_key=None
2 id=206351 version=9380 score=3.9010863 payload={'text': 'In general, what do 

In [10]:
for (idx, value) in zip(sparse_vector_qdrant.indices, sparse_vector_qdrant.values):
    print(f"{model.invert_vocab[idx]}: {value}")

mathematics: 1.0
art: 1.0
science: 1.0
think: 1.0
as: 1.0
?: 1.0
or: 1.0
of: 1.0
you: 1.0
do: 1.0


In [11]:
result_sparse_vector = result[19].vector['attention']

for (idx, value) in zip(result_sparse_vector.indices, result_sparse_vector.values):
    if idx in sparse_vector:
        print(f"{model.invert_vocab[idx]}: {value} x {sparse_vector[idx]} = {value * sparse_vector[idx]}")
    else:
        print(f"{model.invert_vocab[idx]}: {value}")


?: 7.47527e-05 x 1.0 = 7.47527e-05
of: 0.06313381 x 1.0 = 0.06313381
which: 0.1198335
or: 0.12534164 x 1.0 = 0.12534164
use: 0.27951372
less: 0.36276048
science: 0.4092458 x 1.0 = 0.4092458
computer: 0.7822861
engineering: 0.4486358
subject: 0.824075
software: 1.1317307
mathematics: 2.7387416 x 1.0 = 2.7387416
requires: 0.41701674


In [82]:
query = "Is math an art or a science?"
sv = next(model.encode([query]))
sv = conver_sparse_vector(sv)

for (idx, value) in zip(sv.indices, sv.values):
    print(f"{model.invert_vocab[idx]}: {value}")


is: 0.027513518929481506
math: 0.2162715643644333
an: 0.02588239312171936
art: 0.1304435133934021
or: 0.02149391360580921
a: 0.021214833483099937
science: 0.11642558127641678
?: 0.028545841574668884


In [40]:
import math

In [52]:
math.log(1/20 + 1)

0.04879016416943205

In [112]:
a = [1,3,2]

sorted(a, reverse=True)

a

[1, 3, 2]